In [1]:
import os
import cv2
import numpy as np
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import joblib
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report


In [2]:
# Function to extract HOG features from an image
def extract_hog_features(image):
    hog_features = hog(image, pixels_per_cell=(8, 8), cells_per_block=(2, 2), block_norm='L2-Hys')
    return hog_features

# Load the dataset and extract HOG features
def load_data(dataset_path):
    images = []  # List to store HOG features for each image
    labels = []
    class_label_mapping = {
        'Open_Eyes': 0,
        'Closed_Eyes': 1
        # Add more class folders and their corresponding labels if needed
    }
    for class_folder in os.listdir(dataset_path):
        if os.path.isdir(os.path.join(dataset_path, class_folder)):
            class_label = class_label_mapping.get(class_folder)
            if class_label is not None:  # Check if the class folder has a valid label
                class_path = os.path.join(dataset_path, class_folder)
                for image_file in os.listdir(class_path):
                    if image_file.endswith('.png'):  # Assuming all images are in png format
                        image_path = os.path.join(class_path, image_file)
                        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
                        if image is not None:
                            hog_features = extract_hog_features(image)
                            images.append(hog_features.tolist())  # Convert to list before appending
                            labels.append(class_label)
                        else:
                            print(f"Error reading image: {image_path}")
            else:
                print(f"Invalid class folder name: {class_folder}")

    # Find the maximum length of HOG features in the dataset
    max_length = max(len(hog_features) for hog_features in images)

    # Pad the HOG features to the maximum length to make them consistent
    for i in range(len(images)):
        hog_features = images[i]
        pad_length = max_length - len(hog_features)
        images[i] = np.pad(hog_features, (0, pad_length), 'constant')

    return images, np.array(labels)  # Convert labels to a NumPy array

# Load the dataset (provide the path to your dataset folder)
dataset_path = 'data'
images, labels = load_data(dataset_path)
print("Number of samples in the dataset:", len(images))

# Convert images and labels to NumPy arrays
X = np.array(images, dtype=object)  # Convert images list to a NumPy array with dtype=object
y = labels

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Number of samples in the dataset: 4000


In [3]:
# Initialize the SVM model
svm_model = SVC()

# Train the SVM model
svm_model.fit(X_train, y_train)

# Save the model to a file
model_filename = 'svm_model.joblib'
joblib.dump(svm_model, model_filename)

['svm_model.joblib']

In [4]:
# Test the SVM model on the testing set
y_pred = svm_model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Testing Accuracy:", accuracy)

# Calculate precision, recall, and F1-score
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)

# Get the detailed classification report
class_names = ['Open_Eyes', 'Closed_Eyes']  # Replace with your class names if needed
report = classification_report(y_test, y_pred, target_names=class_names)
print("Classification Report:\n", report)

Testing Accuracy: 0.99125
Precision: 0.9859484777517564
Recall: 0.9976303317535545
F1-Score: 0.9917550058892817
Classification Report:
               precision    recall  f1-score   support

   Open_Eyes       1.00      0.98      0.99       378
 Closed_Eyes       0.99      1.00      0.99       422

    accuracy                           0.99       800
   macro avg       0.99      0.99      0.99       800
weighted avg       0.99      0.99      0.99       800

